<a href="https://colab.research.google.com/github/MinsooKwak/RAG/blob/main/test/search/vectordb_reranking_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 유사도 검색 통해 문서 반환시 유사도 높은 순으로 ranking 매겨짐

- RAG는 검색 시간 단축을 위해 Approximate Nearnest Neighbor search 기술을 활용해 </br> 질문과 관계 없는 문서가 검색될 가능성이 높음
- 해결 방안 :
  - 검색으로 반환하는 문서 수 늘리기(토큰 제약으로 불가능)
  - Reranking : 관련 정보가 컨텍스트 내 상위에 존재하도록 함

- 토큰 제약때문에 다 넘길 수 없고, 정말 관련된 문서만 넘겨야

In [1]:
# cohere key (무료이면서 유료(서비스 구현))
# https://dashboard.cohere.com/api-keys

In [2]:
#!pip install cohere
#!pip install langchain
#!pip install openai
#!pip install -U langchain-community
#!pip install pypdf
#!pip install tiktoken
#!pip install faiss-cpu

In [3]:
import os
import openai
from getpass import getpass

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [4]:
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기
loaders = [
    PyPDFLoader("2306.05685v4.pdf")
]

docs = []
for loader in loaders:
  docs.extend(loader.load())

In [5]:
import os
from config import OPEN_AI_API_KEY
os.environ["OPENAI_API_KEY"] = OPEN_AI_API_KEY

chunking

In [6]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
)

# chunking
split_docs = r_splitter.split_documents(docs)
print(split_docs)

[Document(metadata={'source': '2306.05685v4.pdf', 'page': 0}, page_content='Judging LLM-as-a-Judge\nwith MT-Bench and Chatbot Arena\nLianmin Zheng1∗Wei-Lin Chiang1∗Ying Sheng4∗Siyuan Zhuang1\nZhanghao Wu1Yonghao Zhuang3Zi Lin2Zhuohan Li1Dacheng Li13\nEric P. Xing35Hao Zhang12Joseph E. Gonzalez1Ion Stoica1\n1UC Berkeley2UC San Diego3Carnegie Mellon University4Stanford5MBZUAI\nAbstract\nEvaluating large language model (LLM) based chat assistants is challenging due to\ntheir broad capabilities and the inadequacy of existing benchmarks in measuring'), Document(metadata={'source': '2306.05685v4.pdf', 'page': 0}, page_content='their broad capabilities and the inadequacy of existing benchmarks in measuring\nhuman preferences. To address this, we explore using strong LLMs as judges to\nevaluate these models on more open-ended questions. We examine the usage and\nlimitations of LLM-as-a-judge, including position, verbosity, and self-enhancement\nbiases, as well as limited reasoning ability, and

In [7]:
len(split_docs)

249

In [8]:
#split_docs[0].page_content

embedding

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPEN_AI_API_KEY)

<ipython-input-9-2826641ef09d>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPEN_AI_API_KEY)


검색 (reranking 미사용)

In [10]:
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [11]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='gpt-4', api_key=OPEN_AI_API_KEY)

<ipython-input-11-e01b97a28472>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model='gpt-4', api_key=OPEN_AI_API_KEY)


In [12]:
%%time
qa_chain = RetrievalQA.from_chain_type(llm,
                                       chain_type="stuff",
                                       retriever=retriever) # 벡터 검색 결과 같이 넘겨줌

query = "LLM로 평가를 어떻게 진행해?"

qa_chain_result = qa_chain.run(query)
print(qa_chain_result)

<timed exec>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.


LLM을 사용한 평가는 크게 세 가지 방법이 있습니다.

1. 'Pairwise comparison' : LLM 판사에게 하나의 질문과 두 가지 답변이 제시되며, 어느 것이 더 나은지 판단하거나 무승부를 선언하도록 요청합니다.

2. 'Single answer grading' : LLM 판사가 단일 답변에 직접 점수를 부여하도록 요청하는 방법입니다.

3. 'Reference-guided grading' : 특정 경우에는 참조 솔루션을 제공하는 것이 유익할 수 있습니다. 

이 세 가지 방식은 독립적으로 또는 조합하여 구현할 수 있습니다.
CPU times: user 155 ms, sys: 13.9 ms, total: 169 ms
Wall time: 8.1 s


reranking 사용 검색

In [13]:
import os
from config import COHERE_API_KEY
os.environ["COHERE_API_KEY"] = COHERE_API_KEY

In [14]:
compressor = CohereRerank()

# 관련 문서 검색
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever      # 파이쓰 결과
)

compressed_docs = compression_retriever.get_relevant_documents(query)
print(compressed_docs)

<ipython-input-14-584165713aca>:1: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  compressor = CohereRerank()
<ipython-input-14-584165713aca>:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  compressed_docs = compression_retriever.get_relevant_documents(query)


[Document(metadata={'source': '2306.05685v4.pdf', 'page': 3, 'relevance_score': 0.9649213}, page_content='•Pairwise comparison . An LLM judge is presented with a question and two answers, and tasked\nto determine which one is better or declare a tie. The prompt used is given in Figure 5 (Appendix).\n•Single answer grading . Alternatively, an LLM judge is asked to directly assign a score to a\nsingle answer. The prompt used for this scenario is in Figure 6 (Appendix).\n•Reference-guided grading . In certain cases, it may be beneficial to provide a reference solution'), Document(metadata={'source': '2306.05685v4.pdf', 'page': 1, 'relevance_score': 0.9405718}, page_content='our earlier blog post [ 8] and other concurrent or follow-up work [ 5,29,14,12,52,18,33,40,7,43].\nHowever, there has not been a systematic study of this approach.\nIn this paper, we study the LLM-as-a-judge approach by comparing it to the gold standard of\nhuman evaluation. We examine several potential limitations of 

In [15]:
qa_chain_reranked = RetrievalQA.from_chain_type(llm,
                                       chain_type="stuff",
                                       retriever=compression_retriever)

In [16]:
%%time
qa_chain_rerank_result = qa_chain_reranked.run(query)
print(qa_chain_rerank_result)

LLM 평가는 세 가지 방법으로 진행할 수 있습니다.

1. Pairwise comparison: LLM 판사에게 질문과 두 가지 답변이 제시되며, 어느 것이 더 나은지 판단하거나 무승부를 선언하는 작업을 수행합니다.
   
2. Single answer grading: 다른 방법으로는, LLM 판사에게 하나의 답변에 직접 점수를 부여하도록 요청할 수 있습니다.

3. Reference-guided grading: 특정 경우에는 참조 솔루션을 제공하는 것이 유익할 수 있습니다.

이 세 가지 방법은 독립적으로 또는 결합하여 구현할 수 있습니다.
CPU times: user 105 ms, sys: 5.31 ms, total: 110 ms
Wall time: 7.5 s


결과 비교

In [17]:
print(qa_chain_result)
print('-'*70)
print(qa_chain_rerank_result)

LLM을 사용한 평가는 크게 세 가지 방법이 있습니다.

1. 'Pairwise comparison' : LLM 판사에게 하나의 질문과 두 가지 답변이 제시되며, 어느 것이 더 나은지 판단하거나 무승부를 선언하도록 요청합니다.

2. 'Single answer grading' : LLM 판사가 단일 답변에 직접 점수를 부여하도록 요청하는 방법입니다.

3. 'Reference-guided grading' : 특정 경우에는 참조 솔루션을 제공하는 것이 유익할 수 있습니다. 

이 세 가지 방식은 독립적으로 또는 조합하여 구현할 수 있습니다.
----------------------------------------------------------------------
LLM 평가는 세 가지 방법으로 진행할 수 있습니다.

1. Pairwise comparison: LLM 판사에게 질문과 두 가지 답변이 제시되며, 어느 것이 더 나은지 판단하거나 무승부를 선언하는 작업을 수행합니다.
   
2. Single answer grading: 다른 방법으로는, LLM 판사에게 하나의 답변에 직접 점수를 부여하도록 요청할 수 있습니다.

3. Reference-guided grading: 특정 경우에는 참조 솔루션을 제공하는 것이 유익할 수 있습니다.

이 세 가지 방법은 독립적으로 또는 결합하여 구현할 수 있습니다.
